In [1]:
# https://androidkt.com/use-saved-pytorch-model-to-predict-single-and-multiple-images/
# https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-predict-new-samples-with-your-pytorch-model.mdS
# https://towardsdatascience.com/optimize-pytorch-performance-for-speed-and-memory-efficiency-2022-84f453916ea6
# https://www.kaggle.com/code/andradaolteanu/pytorch-rnns-and-lstms-explained-acc-0-99#3.-RNN-with-1-Layer-%F0%9F%93%98
# https://www.kaggle.com/code/omershect/learning-pytorch-lstm-deep-learning-with-m5-data/comments
# https://curiousily.com/posts/time-series-anomaly-detection-using-lstm-autoencoder-with-pytorch-in-python/
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import time
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Available Device is: ', device)

Available Device is:  cpu


In [3]:
df = pd.read_csv('./robotdatacollection3/ep2.csv')
feature_list = ['Fx','Fy','Fz','Mx','My']
TIMESTEP = 50

In [4]:
df_new = df[feature_list]
print('Shape before adding dimension:',df_new.shape)
df_new = np.expand_dims(df_new, axis=1)
print('Shape AFTER adding dimension:',df_new.shape)

Shape before adding dimension: (5351, 5)
Shape AFTER adding dimension: (5351, 1, 5)


In [5]:
print(f"""LSTM input has format of:
    sample number = {df_new.shape[0]} 
    window size = {df_new.shape[1]} 
    feature number= {df_new.shape[2]}""")

LSTM input has format of:
    sample number = 5351 
    window size = 1 
    feature number= 5


In [6]:
df_new.shape

(5351, 1, 5)

### Sequence

In [7]:
# we set the data in a window form, in case of timestep = 5
# i=0: X_0=[x(0)..x(5)], y=x(6)

def to_sequence(data, timesteps=1):
    n_features=data.shape[2]
    x = []
    y = []
    for i in range(len(data)-timesteps):
        # takes a window of data of specified timesteps
        
        _x = data[i:(i+timesteps)]
        _x = _x.reshape(timesteps, n_features)
#         print(_x.shape)
        _y = data[i+timesteps]
        _y = _y.reshape(n_features)
#         print(_y.shape)
        x.append(_x)
        y.append(_y)

        
    return np.array(x), np.array(y)

In [8]:
x_train, y_train = to_sequence(df_new, timesteps=TIMESTEP)

In [9]:
print(x_train.shape)
print(y_train.shape)

(5301, 50, 5)
(5301, 5)


**Pytorch uses Tensors as inputs to the model, so we convert numpy to Tensor**

In [10]:
trainX = torch.Tensor(x_train)
trainy = torch.Tensor(y_train)

In [11]:
print(trainX.shape)
print(trainy.shape)

torch.Size([5301, 50, 5])
torch.Size([5301, 5])


### LSTM


LSTM layers work on 3D data with the following structure `(sequence/batch, timestep, feature)`.

* **input_dim**: Number of samples in the data. In our case we do not use batches but instead one sample at a time. Each sample has TIMESTEP size of data in it. 
* **hidden_dim**: Number of LSTM cells
* **layer_dim**: Number of LSTM hidden layers
* **output_dim**: Number of outputs = feature number (in case of regression task)




In [12]:
#Bidirectional LSTM: extensions that can improve performance. They train model forward and backward on the same input
# much slower, good for NLP 

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__() # inherit from nn.Module
        
        # hidden dims
        self.hidden_dim = hidden_dim
        # number of hidden layers
        self.layer_dim = layer_dim
        
        # building LSTM
        # setting batch_first=True: input and output order: (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        
        # output layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
        
    def forward(self, x):
#         print('input shape',x.shape)
        # initialize hidden state with zeros
        hidden_state = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()
        # initialize cell state
        cell_state = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()
    
        
        # we need to detach tensor from current graph
        # if we do not, we will backpop all the way to the start even after going through another batch
        out, (last_hidden_state, last_cell_state) = self.lstm(x, (hidden_state.detach(), cell_state.detach()))
#         print('output shape', out.shape)
        
        # index hidden state of last time step
        # out.size() -> 100,28,100
        # out[:,-1,:] -> 100, 100-> just want last time step hidden states
        
        # rehape
        out = self.fc(out[:,-1,:])
#         print('reshape',out.shape)
        # out.size() --> 100,10
        return out

In [13]:
input_dim = 5# 50 # timestep
hidden_dim = 15 # LSTM layer 5 units
layer_dim = 1 # one LSTM layer
output_dim = 5 # we outout one prediction at a time

In [14]:
lstm_model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
# lstm_model.to(device)

In [15]:
lstm_model

LSTMModel(
  (lstm): LSTM(5, 15, batch_first=True)
  (fc): Linear(in_features=15, out_features=5, bias=True)
)

In [16]:
# regression, calculates the loss function
criterion = torch.nn.MSELoss()
# updates the weights and biases to reduce loss
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=0.001)


# Training LSTM for each epoch we
for epoch in range(100):
    
    # enable train mode
    lstm_model.train(True)
    
    # clear gradients with respect to params, always before backprop
    optimizer.zero_grad(set_to_none=True)
    
    # track history if only in train
    with torch.set_grad_enabled(True):
        
        # forward pass to get outputs/ make predictions for that batch
        output = lstm_model(trainX)
        # calculate loss and its gradients
        loss = criterion(output, trainy)
        # getting gradients w.r.t. params
        loss.backward()

        #updating params/ weights
        optimizer.step()
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss {loss}")
    

Epoch 0, Loss 6.034580707550049
Epoch 10, Loss 5.793862819671631
Epoch 20, Loss 5.539210319519043
Epoch 30, Loss 5.250224590301514
Epoch 40, Loss 4.906777858734131
Epoch 50, Loss 4.503946304321289
Epoch 60, Loss 4.058081150054932
Epoch 70, Loss 3.5879971981048584
Epoch 80, Loss 3.100661039352417
Epoch 90, Loss 2.6243836879730225


In [17]:
test_x = np.ones((1,50,5))
test_x = torch.Tensor(test_x)
print(test_x.shape)
print()

t = 0
times_dnn = []
lstm_model.eval()
torch.no_grad()

while t < 10:
    t_start = time.time()
    
    with torch.no_grad():
        y_pred = lstm_model(test_x)
        print(y_pred)
    dt = time.time()-t_start
    print('dt',dt)
    
    times_dnn.append(dt)
    t += 1

torch.Size([1, 50, 5])

tensor([[-0.2265,  0.4771,  1.4639, -0.0226,  0.1733]])
dt 0.002825021743774414
tensor([[-0.2265,  0.4771,  1.4639, -0.0226,  0.1733]])
dt 0.0012857913970947266
tensor([[-0.2265,  0.4771,  1.4639, -0.0226,  0.1733]])
dt 0.0009996891021728516
tensor([[-0.2265,  0.4771,  1.4639, -0.0226,  0.1733]])
dt 0.0008528232574462891
tensor([[-0.2265,  0.4771,  1.4639, -0.0226,  0.1733]])
dt 0.0007534027099609375
tensor([[-0.2265,  0.4771,  1.4639, -0.0226,  0.1733]])
dt 0.0007054805755615234
tensor([[-0.2265,  0.4771,  1.4639, -0.0226,  0.1733]])
dt 0.0006396770477294922
tensor([[-0.2265,  0.4771,  1.4639, -0.0226,  0.1733]])
dt 0.0006031990051269531
tensor([[-0.2265,  0.4771,  1.4639, -0.0226,  0.1733]])
dt 0.0005667209625244141
tensor([[-0.2265,  0.4771,  1.4639, -0.0226,  0.1733]])
dt 0.0005071163177490234


In [18]:
test_x = np.ones((1,1,5))
test_x = torch.Tensor(test_x)
print(test_x.shape)
print()

t = 0
times_dnn = []
lstm_model.eval()
torch.no_grad()

while t < 10:
    t_start = time.time()
    
    with torch.no_grad():
        y_pred = lstm_model(test_x)
        print(y_pred)
    dt = time.time()-t_start
    print('dt',dt)
    
    times_dnn.append(dt)
    t += 1

torch.Size([1, 1, 5])

tensor([[-0.1676,  0.2939, -0.0378,  0.0750,  0.0078]])
dt 0.0020952224731445312
tensor([[-0.1676,  0.2939, -0.0378,  0.0750,  0.0078]])
dt 0.0003695487976074219
tensor([[-0.1676,  0.2939, -0.0378,  0.0750,  0.0078]])
dt 0.0002911090850830078
tensor([[-0.1676,  0.2939, -0.0378,  0.0750,  0.0078]])
dt 0.0002658367156982422
tensor([[-0.1676,  0.2939, -0.0378,  0.0750,  0.0078]])
dt 0.00025153160095214844
tensor([[-0.1676,  0.2939, -0.0378,  0.0750,  0.0078]])
dt 0.00022935867309570312
tensor([[-0.1676,  0.2939, -0.0378,  0.0750,  0.0078]])
dt 0.0002288818359375
tensor([[-0.1676,  0.2939, -0.0378,  0.0750,  0.0078]])
dt 0.0004115104675292969
tensor([[-0.1676,  0.2939, -0.0378,  0.0750,  0.0078]])
dt 0.0002067089080810547
tensor([[-0.1676,  0.2939, -0.0378,  0.0750,  0.0078]])
dt 0.00020003318786621094
